### Import necessary libraries

In [ ]:
import pandas as pd
import numpy as np
import cv2
import folium
import os
from openpyxl import load_workbook
from folium.plugins import TimestampedGeoJson
from datetime import datetime

### Process the mask

In [64]:
def process_image(image):
    # map of color to label
    color_to_label = {
        (0, 0, 255): 'harbor',  # BGR for red
        (0, 255, 0): 'jetty',   # BGR for green
        (255, 0, 0): 'resort'   # BGR for blue
    }

    results = []

    # Traverse each color mask to extract landmark information
    for color, label in color_to_label.items():
        lower = np.array(color, dtype="uint8")
        upper = np.array(color, dtype="uint8")
        mask = cv2.inRange(image, lower, upper)
        
        kernel = np.ones((3, 3), np.uint8)
        mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)

        # perform connected component analysis on the mask
        num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(mask, connectivity=8)

        # collect label, centroid and area of each connected component
        for i in range(1, num_labels):  
            x, y = centroids[i]
            area = stats[i, cv2.CC_STAT_AREA]
            results.append({
                'label': label,
                'centroid': (int(x), int(y)),
                'area': int(area)
            })

    return results

In [126]:
# folder path 
folder_path = 'predicted_mask/Meedhoo_3'

# acquire all image files in the folder
image_files = [f for f in os.listdir(folder_path) if f.endswith('.png')]

# initialize a DataFrame to store the results
columns = ['date', 'detected_label', 'true_label', 'centroid_x', 'centroid_y', 'area']
df = pd.DataFrame(columns=columns)

# Iterate over each image file
for image_file in image_files:
    # if file name is in the format of 'Meedhoo_1_2021-01-01.png'
    # use: date = image_file.split('_')[2]
    # if file name is in the format of 'Meedhoo_2021-01-01.png'
    # use: date = image_file.split('_')[1]
    date = image_file.split('_')[2]
    image_path = os.path.join(folder_path, image_file)
    image = cv2.imread(image_path)
    processed_results = process_image(image)  

    # If no landmarks are detected (black mask), add a NaN entry
    if not processed_results:
        new_row = {
            'date': date,
            'detected_label': np.nan,
            'true_label': np.nan,
            'centroid_x': np.nan,
            'centroid_y': np.nan,
            'area': np.nan
        }
        df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)
    else:
        # Add date, detected label, and true label information
        for result in processed_results:
            new_row = {
                'date': date,
                'detected_label': result['label'],
                'true_label': np.nan,  # initially set to NaN
                'centroid_x': result['centroid'][0],
                'centroid_y': result['centroid'][1],
                'area': result['area']
            }
            df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)

In [127]:
df

,date,detected_label,true_label,centroid_x,centroid_y,area
0,2017-09-09,harbor,NaN,154,87,1019
1,2017-09-09,resort,NaN,60,88,44
2,2017-07-11,harbor,NaN,154,78,640
3,2020-10-28,harbor,NaN,136,139,320
4,2017-06-26,harbor,NaN,160,78,399
5,2020-12-12,harbor,NaN,163,89,52
6,2020-12-12,harbor,NaN,142,141,393
7,2020-08-14,harbor,NaN,152,91,364
8,2020-08-14,harbor,NaN,142,149,228
9,2018-04-07,harbor,NaN,159,90,931


### Convert pixel coordinates to latitude and longitude coordinates

In [128]:
def calculate_lat_lon(top_left_lat, top_left_lon, bottom_right_lat, bottom_right_lon, 
                      original_width, original_height, i_prime, j_prime, 
                      resized_width=224, resized_height=224):

    # Map the pixel position from the resized image to the original image
    i = (i_prime / (resized_height - 1)) * (original_height - 1)
    j = (j_prime / (resized_width - 1)) * (original_width - 1)
    
    # Calculate the latitude and longitude for the pixel position in the original image
    lat_range = bottom_right_lat - top_left_lat
    lon_range = bottom_right_lon - top_left_lon
    
    lat = top_left_lat + (i / (original_height - 1)) * lat_range
    lon = top_left_lon + (j / (original_width - 1)) * lon_range
    
    return lat, lon


In [129]:
top_left_lat, top_left_lon = -0.594651, 73.217103
bottom_right_lat, bottom_right_lon = -0.600996, 73.225116
original_width, original_height = 90, 72

# Apply the conversion to each row in the DataFrame
df[['latitude', 'longitude']] = df.apply(
    lambda row: calculate_lat_lon(top_left_lat, top_left_lon, bottom_right_lat, bottom_right_lon, 
                                  original_width, original_height, row['centroid_x'], row['centroid_y'], 
                                  ), axis=1, result_type='expand')

In [130]:
df.sort_values(by=['date'], inplace=True)
df

,date,detected_label,true_label,centroid_x,centroid_y,area,latitude,longitude
25,2016-03-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,2017-03-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,2017-04-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17,2017-06-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,2017-06-16,harbor,NaN,159,79,281,-0.599175,73.219942
4,2017-06-26,harbor,NaN,160,78,399,-0.599203,73.219906
2,2017-07-11,harbor,NaN,154,78,640,-0.599033,73.219906
0,2017-09-09,harbor,NaN,154,87,1019,-0.599033,73.220229
1,2017-09-09,resort,NaN,60,88,44,-0.596358,73.220265
12,2017-10-19,harbor,NaN,157,87,905,-0.599118,73.220229


### Save DataFrame to an Excel file

In [30]:
# The code is only used for the first time to create the Excel file
file_path = "original_result.xlsx"
df.to_excel(file_path, sheet_name="Sheet1", index=False)

### Append a DataFrame to the existing Excel file

In [131]:
file_path = "original_result.xlsx"

with pd.ExcelWriter(file_path, engine="openpyxl", mode='a', if_sheet_exists='overlay') as writer:
    writer.book = load_workbook(file_path)
    startrow = writer.book["Sheet1"].max_row
    df.to_excel(writer, sheet_name="Sheet1", startrow=startrow, index=False, header=False)

/var/folders/b3/k0w8k2kd1s78zzp67h0d95900000gp/T/ipykernel_54668/3504172395.py:4: FutureWarning: Setting the `book` attribute is not part of the public API, usage can give unexpected or corrupted results and will be removed in a future version
  writer.book = load_workbook(file_path)


### Folium Map Visualization 

In [3]:
# read csv
file_path = 'final_result.xlsx'
df = pd.read_excel(file_path)

# ensure date column is in string format
df['date'] = df['date'].fillna('Unknown').astype(str)

# deal with date range
def parse_date_range(date_str):
    if '-' in date_str:
        start, end = date_str.split('-')
        start = pd.to_datetime(start, format='%Y.%m')
        if end.lower() == "present":
            end = datetime.now()
        else:
            end = pd.to_datetime(end, format='%Y.%m')
    elif date_str.lower() == "unknown":
        start = end = pd.NaT  
    else:
        start = pd.to_datetime(date_str, format='%Y.%m')
        end = start
    return start, end

df['date_start'], df['date_end'] = zip(*df['date'].apply(parse_date_range))

df['date_end'] = df['date_end'].fillna(datetime.now())

# sort by date_start
df = df.sort_values(by='date_start')

m = folium.Map(location=[df['latitude'].mean(), df['longitude'].mean()], zoom_start=14, tiles='CartoDB positron')

# define different colors for different status
status_colors = {
    'construction completed': 'green',
    'continuously expanding': 'blue',
    'demolished': 'black',
    'existing': 'gray',
    'expansion completed': 'purple',
    'newly constructed': 'orange',
    'non-existent': 'red',
    'under construction': '#008080',
}

# define different shapes for different location types
location_shapes = {
    'harbor': {'radius': 7, 'fillOpacity': 1, 'weight': 2, 'fillColor': None},
    'jetty': {'radius': 3, 'fillOpacity': 0.5, 'weight': 2, 'fillColor': None},
    'resort': {'radius': 5, 'fillOpacity': 0, 'weight': 2, 'fillColor': None}
}

# create geojson features
features = []
for _, row in df.iterrows():
    status = row['status']
    color = status_colors.get(status, 'gray')
    shape_attr = location_shapes[row['true_label'].lower()]
    
    # create popup content
    popup_content = f"Label: {row['true_label']}<br>Status: {row['status']}<br>Date: {row['date']}"

    # show the status in the popup content
    feature = {
        'type': 'Feature',
        'geometry': {
            'type': 'Point',
            'coordinates': [row['longitude'], row['latitude']],
        },
        'properties': {
            'times': [row['date_start'].strftime('%Y-%m-%d'), row['date_end'].strftime('%Y-%m-%d')],
            'popup': popup_content,
            'style': {
                'radius': shape_attr['radius'],
                'fillOpacity': shape_attr['fillOpacity'],  # 填充不透明度
                'fillColor': color if shape_attr['fillOpacity'] > 0 else None,  # 根据填充透明度设置填充颜色
                'stroke': True,
                'color': color,  # 轮廓使用状态颜色
                'weight': shape_attr['weight'],  # 轮廓线宽
            },
            'icon': 'circle'  # 设置形状为圆形
        }
    }
    features.append(feature)

# create TimestampedGeoJson object
timestamped_geojson = TimestampedGeoJson(
    {
        'type': 'FeatureCollection',
        'features': features,
    },
    period='P1M',  
    add_last_point=True,
    auto_play=False,
    loop=False,
    max_speed=1,
    loop_button=True,
    date_options='YYYY-MM-DD',
    time_slider_drag_update=True,
)

timestamped_geojson.add_to(m)

# add legend
legend_html = """
<div style="position: fixed; 
            bottom: 50px; left: 50px; width: 220px; height: auto; 
            border:2px solid grey; z-index:9999; font-size:12px;
            background-color:white;
            opacity: 0.8;
            padding: 5px 10px; margin: 0;">
<h4 style="margin-top: 0; margin-bottom: 5px;">Status Legend</h4>
<i style="background-color: gray; width: 12px; height: 12px; display: inline-block; margin-right: 5px;"></i> Existing<br>
<i style="background-color: red; width: 12px; height: 12px; display: inline-block; margin-right: 5px;"></i> Non-existent<br>
<i style="background-color: orange; width: 12px; height: 12px; display: inline-block; margin-right: 5px;"></i> Newly Constructed<br>
<i style="background-color: black; width: 12px; height: 12px; display: inline-block; margin-right: 5px;"></i> Demolished<br>
<i style="background-color: blue; width: 12px; height: 12px; display: inline-block; margin-right: 5px;"></i> Continuously Expanding<br>
<i style="background-color: purple; width: 12px; height: 12px; display: inline-block; margin-right: 5px;"></i> Expansion Completed<br>
<i style="background-color: #008080; width: 12px; height: 12px; display: inline-block; margin-right: 5px;"></i> Under Construction<br>
<i style="background-color: green; width: 12px; height: 12px; display: inline-block; margin-right: 5px;"></i> Construction Completed<br>

<h4 style="margin-top: 10px; margin-bottom: 5px;">Location Types</h4>
<svg width="12" height="12" style="margin-right: 5px;"><circle cx="6" cy="6" r="6" fill="black" stroke="black" stroke-width="2"></circle></svg> Harbor (Fully Filled)<br>
<svg width="12" height="12" style="margin-right: 5px;"><circle cx="6" cy="6" r="6" fill="black" fill-opacity="0.5" stroke="black" stroke-width="2"></circle></svg> Jetty (Half Filled)<br>
<svg width="12" height="12" style="margin-right: 5px;"><circle cx="6" cy="6" r="6" fill="none" stroke="black" stroke-width="2"></circle></svg> Resort (No Fill)<br>
</div>
"""


m.get_root().html.add_child(folium.Element(legend_html))


# save the map
m.save('map_with_timeline.html')

In [8]:
m